In [1]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle

# Carregar e preparar os dados de treino
train_data = pd.read_csv('imdb_data_train/imdb_data_train')
reviews = train_data['review'].tolist()
sentiments = train_data['sentiment'].tolist()

# Função de limpeza e tokenização
def clean_tokenize(document):
    document = re.sub('[^a-zA-Z]', ' ', document)
    tokens = word_tokenize(document.lower())
    cleaned_review = [word for word in tokens if word not in stopwords.words('english')]
    return cleaned_review

# Preparar os dados de treino
cleaned_data = [" ".join(clean_tokenize(review)) for review in reviews]
tfidfconverter = TfidfVectorizer(max_features=1500)
X_train = tfidfconverter.fit_transform(cleaned_data).toarray()
y_train = sentiments

# Inicializar e preparar os dados de teste
test_data = pd.read_csv('imdb_data_test/imdb_data_test')
test_reviews = test_data['review'].tolist()
test_sentiments = test_data['sentiment'].tolist()

# Preparar os dados de teste
cleaned_test_data = [" ".join(clean_tokenize(review)) for review in test_reviews]
X_test = tfidfconverter.transform(cleaned_test_data).toarray()
y_test = test_sentiments

# Inicializar o GridSearchCV
param_grid = {
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Treinar o modelo com GridSearch
grid_search.fit(X_train, y_train)

# Melhores parâmetros e modelo
print("Melhores parâmetros: ", grid_search.best_params_)
best_classifier = grid_search.best_estimator_

# Fazer previsões e avaliar
y_pred = best_classifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Salvar o modelo e o vetorizador
pickle.dump(best_classifier, open('Decision_Tree_model.pkl', 'wb'))


Fitting 5 folds for each of 27 candidates, totalling 135 fits
Melhores parâmetros:  {'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 2}
[[ 8320  4180]
 [ 2479 10021]]
              precision    recall  f1-score   support

           0       0.77      0.67      0.71     12500
           1       0.71      0.80      0.75     12500

    accuracy                           0.73     25000
   macro avg       0.74      0.73      0.73     25000
weighted avg       0.74      0.73      0.73     25000
